In [1]:
#!/usr/bin/env python
# Extract fluxnet and micasa data as csv for plotting
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import glob
import argparse
import sys
import os
import pytz
from timezonefinder import TimezoneFinder

In [2]:
######### functions ############
def get_single_match(pattern):
    matches = glob.glob(pattern)
    if len(matches) == 1:
        return matches[0]
    elif len(matches) == 0:
        raise ValueError("No matches found")
    else:
        raise ValueError(f"Multiple matches found: {matches}")

In [3]:
site_ID = 'US-Ne1'
timedelta = 'DD'
micasa_var = 'NEE'

In [6]:
# Open site ID metadata and extract lat/lon
filepath = '../ameriflux-data/'
meta_file = filepath + 'AmeriFlux-site-search-results-202410071335.tsv'
ameriflux_meta = pd.read_csv(meta_file, sep='\t')
site_lat = ameriflux_meta.loc[ameriflux_meta['Site ID'] == site_ID, 'Latitude (degrees)'].values
site_lon = ameriflux_meta.loc[ameriflux_meta['Site ID'] == site_ID, 'Longitude (degrees)'].values

In [ ]:
# # Test one path
# path_list = ['micasa-data/daily-0.1deg-final/holding/daily/2001/01/MiCASA_v1_flux_x3600_y1800_daily_20010120.nc4']

# with xr.open_mfdataset(path_list)[micasa_var] as ds:
#     # Select grid closest to selected site
#     ds_subset = ds.sel(lon=site_lon, lat=site_lat, method='nearest')

#     # Prep data for writing to csv
#     ds_out = ds_subset.squeeze(dim=['lat','lon'],drop=True).to_dataframe()
#     ds_out = ds_out.rename(columns={micasa_var: f'MiCASA {micasa_var} ({test_ds[micasa_var].units})'})

#     # Write to csv
#     output_dir = 'output'
#     output_filename = f'{site_ID}_micasa_{micasa_var}_{timedelta}.csv'
#     output_path = os.path.join(output_dir, output_filename)

#     os.makedirs(output_dir, exist_ok=True)

#     ds_out.to_csv(output_path)


In [12]:
test_ds = xr.open_dataset('../micasa-data/daily-0.1deg-final/holding/daily/2001/01/MiCASA_v1_flux_x3600_y1800_daily_20010120.nc4')
test_ds

<xarray.Dataset> Size: 156MB
Dimensions:    (lat: 1800, lon: 3600, time: 1, nv: 2)
Coordinates:
  * lat        (lat) float64 14kB -89.95 -89.85 -89.75 ... 89.75 89.85 89.95
  * lon        (lon) float64 29kB -179.9 -179.8 -179.8 ... 179.8 179.8 179.9
  * time       (time) datetime64[ns] 8B 2001-01-20
Dimensions without coordinates: nv
Data variables:
    time_bnds  (time, nv) datetime64[ns] 16B ...
    NPP        (time, lat, lon) float32 26MB ...
    Rh         (time, lat, lon) float32 26MB ...
    FIRE       (time, lat, lon) float32 26MB ...
    FUEL       (time, lat, lon) float32 26MB ...
    ATMC       (time, lat, lon) float32 26MB ...
    NEE        (time, lat, lon) float32 26MB ...

In [18]:
test_ds = test_ds.sel(lon=site_lon, lat=site_lat, method='nearest')
test_ds

<xarray.Dataset> Size: 64B
Dimensions:    (lat: 1, lon: 1, time: 1, nv: 2)
Coordinates:
  * lat        (lat) float64 8B 41.15
  * lon        (lon) float64 8B -96.45
  * time       (time) datetime64[ns] 8B 2001-01-20
Dimensions without coordinates: nv
Data variables:
    time_bnds  (time, nv) datetime64[ns] 16B 2001-01-20 2001-01-21
    NPP        (time, lat, lon) float32 4B ...
    Rh         (time, lat, lon) float32 4B ...
    FIRE       (time, lat, lon) float32 4B ...
    FUEL       (time, lat, lon) float32 4B ...
    ATMC       (time, lat, lon) float32 4B ...
    NEE        (time, lat, lon) float32 4B ...

In [22]:
test_ds_out = test_ds.squeeze(dim=['lat','lon'],drop=True)
test_ds_out[micasa_var]

<xarray.DataArray 'NEE' (time: 1)> Size: 4B
[1 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 8B 2001-01-20
Attributes:
    units:         kg m-2 s-1
    expressed_as:  carbon
    long_name:     Net ecosystem exchange

In [28]:
test_ds_out['NPP']

<xarray.DataArray 'NPP' (time: 1)> Size: 4B
[1 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 8B 2001-01-20
Attributes:
    units:         kg m-2 s-1
    expressed_as:  carbon
    long_name:     Net primary productivity

In [23]:
test_ds_out[micasa_var].units

'kg m-2 s-1'

In [ ]:
column_name = f'MiCASA {micasa_var} ({test_ds[micasa_var].units})'
column_name